In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folders = glob.glob('/content/drive/MyDrive/ai_project/train/*')
all_img_list = glob.glob('/content/drive/MyDrive/ai_project/train/*/*')
root = '/content/drive/MyDrive/ai_project/train'
test_folder = '/content/drive/MyDrive/ai_project/test'

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [ ]:
res_transform = A.Compose([
                          A.Resize(224,224),
                          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                          ToTensorV2()
                          ])

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
savepath_c0 = '/content/drive/MyDrive/ai_project/5.18/small_100_5_18_21:57:56.pt'
savepath_c1 = '/content/drive/MyDrive/ai_project/5.18/small_1_100_5_18_23_7_55.pt'
# savepath_c1 = '/content/drive/MyDrive/ai_project/5.18/samll_100_5_18_22_1_26.pt'
savepath_c2 = '/content/drive/MyDrive/ai_project/5.18/small_2_100_5_18_22_10_14.pt'
savepath_c3 = '/content/drive/MyDrive/ai_project/5.18/samll_100_5_18_22_20_58.pt'
savepath_sub = '/content/drive/MyDrive/ai_project/5.18/big_100_5_18_23:39:6.pt'
savepath_list = [savepath_c0,savepath_c1,savepath_c2,savepath_c3,savepath_sub]

In [ ]:
model_c0 = BaseModel(num_classes = 4)
model_c1 = BaseModel(num_classes = 7)
model_c2 = BaseModel(num_classes = 5)
model_c3 = BaseModel(num_classes = 3)
model_sub = BaseModel(num_classes = 4)
model_list = [model_c0,model_c1,model_c2,model_c3,model_sub]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for i in range(len(model_list)):
  model_list[i].load_state_dict(torch.load(savepath_list[i]))
  model_list[i].to(device)
  model_list[i].eval()


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
test_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+test['img_path'].values, None, res_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)

In [ ]:
def label_decoder0(label):
    new_label =[
        '곰팡이' if l==0 else
        '녹오염' if l==1 else
        '반점' if l==2 else
        '오염' if l==3 else
        'NaN' for l in label
    ]
    return np.array(new_label)
def label_decoder1(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '꼬임' if l==2 else
        '들뜸' if l==3 else
        '몰딩수정' if l==4 else
        '창틀,문틀수정' if l==5 else
        '틈새과다' if l==6 else
        'NaN' for l in label
    ]
    return np.array(new_label)
def label_decoder2(label):
    new_label =[
        '면불량' if l==0 else
        '울음' if l==1 else
        '이음부불량' if l==2 else
        '터짐' if l==3 else
        '훼손' if l==4 else
        'NaN' for l in label
    ]
    return np.array(new_label)
def label_decoder3(label):
    new_label =[
        '석고수정' if l==0 else
        '오타공' if l==1 else
        '피스' if l==2 else
        'NaN' for l in label
    ]
    return np.array(new_label)
decoder_list = [label_decoder0,label_decoder1,label_decoder2,label_decoder3]

In [ ]:
def inference(model, res_loader, device):
    model.eval()
    preds = []
    pred_kor=[]
    with torch.no_grad():
        for imgs in tqdm(iter(res_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            if preds[-1] == 0:
              pred = model_c0(imgs)
              pred_kor.append(decoder_list[0](pred.argmax(1).detach().cpu().numpy().tolist())[0])
            elif preds[-1] == 1:
              pred = model_c1(imgs)
              pred_kor.append(decoder_list[1](pred.argmax(1).detach().cpu().numpy().tolist())[0])
            elif preds[-1] == 2:
              pred = model_c2(imgs)
              pred_kor.append(decoder_list[2](pred.argmax(1).detach().cpu().numpy().tolist())[0])
            elif preds[-1] == 3:
              pred = model_c3(imgs)
              pred_kor.append(decoder_list[3](pred.argmax(1).detach().cpu().numpy().tolist())[0])
    # preds = le.inverse_transform(preds)
    print(pred_kor)
    return pred_kor

In [ ]:
preds = inference(model_sub, test_loader, device)
preds = [str(x) for x in preds]

  0%|          | 0/792 [00:00<?, ?it/s]

['훼손', '녹오염', '터짐', '몰딩수정', '면불량', '녹오염', '곰팡이', '녹오염', '곰팡이', '오타공', '곰팡이', '피스', '이음부불량', '오염', '곰팡이', '면불량', '석고수정', '몰딩수정', '터짐', '반점', '창틀,문틀수정', '곰팡이', '면불량', '오염', '창틀,문틀수정', '면불량', '피스', '꼬임', '훼손', '면불량', '면불량', '훼손', '오염', '훼손', '피스', '곰팡이', '녹오염', '울음', '오타공', '오타공', '면불량', '들뜸', '훼손', '꼬임', '터짐', '녹오염', '면불량', '오염', '훼손', '면불량', '면불량', '터짐', '훼손', '녹오염', '오타공', '오염', '오염', '피스', '면불량', '터짐', '피스', '몰딩수정', '들뜸', '이음부불량', '몰딩수정', '오염', '면불량', '걸레받이수정', '면불량', '피스', '오염', '걸레받이수정', '면불량', '면불량', '오타공', '터짐', '곰팡이', '훼손', '오염', '오타공', '면불량', '면불량', '오염', '피스', '곰팡이', '오타공', '곰팡이', '터짐', '창틀,문틀수정', '곰팡이', '면불량', '꼬임', '터짐', '곰팡이', '훼손', '곰팡이', '피스', '피스', '오타공', '오염', '터짐', '오염', '곰팡이', '면불량', '반점', '몰딩수정', '가구수정', '오타공', '훼손', '곰팡이', '오염', '오타공', '면불량', '오염', '면불량', '터짐', '훼손', '터짐', '오염', '터짐', '곰팡이', '터짐', '몰딩수정', '녹오염', '곰팡이', '훼손', '면불량', '곰팡이', '면불량', '오염', '피스', '이음부불량', '오염', '면불량', '오타공', '훼손', '녹오염', '오염', '훼손', '오염', '울음', '피스', '곰팡이', '터짐', '피스', '울음', '오타공', '녹오염', 

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ai_project/sample_submission.csv',encoding='utf-8-sig')
submit['label'] = preds
submit.to_csv('/content/drive/MyDrive/ai_project/5.18_1.csv', index=False, encoding='utf-8-sig')